In [3]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
from betting_functions import get_url_json

In [4]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
data = get_url_json(dog_props_url)

In [5]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [6]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
1749,1a55c232-6f1f-49c5-b00f-0db09087ffcb,False,None,[{'id': '9eb17177-8ae1-460a-a139-96fdb431ec47'...,7e0ca1e8-76ac-4e2f-aa0d-58106529a462,25101149,2.5,active,7e0ca1e8-76ac-4e2f-aa0d-58106529a462,ac246a02-3ea7-41fc-82fa-336d4f0d7784,b6a731b3-ab5a-43a9-912f-f31fad6519c9,Deaths on Map 3,high_score,e37fa97b-a4a3-4e45-9e49-1735663bc19e,deaths_in_map_3,None,None,LoL: Xiaohu Deaths on Map 3 O/U
1984,c9de169e-b978-4761-9af3-cfc130f1c545,False,None,[{'id': '2c564d7c-8d58-490d-b2a9-4dc34a015aba'...,704147b2-5839-422d-87b9-1c818ec57adb,25121151,24.0,active,704147b2-5839-422d-87b9-1c818ec57adb,10f2c7fb-1894-4c7d-9325-95a6c661cb3d,9780411a-4fcd-4143-a520-beb60adab98b,Kills on Map 1+2,high_score,fd4d222d-d4d0-4258-b0d2-d225e1947b0c,map_1_and_2_kills,None,None,Val: Vanity Kills on Map 1+2 O/U


In [7]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [8]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [9]:
player_df

,full_name,sport_id,team_id
0,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
1,Miles Sanders,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
2,Chuba Hubbard,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
3,Adam Thielen,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
4,Jonathan Mingo,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
...,...,...,...
761,A. Thielen + J. Mingo,NFLCOMBO,7d44628a-8429-41dd-a7d3-a631a8d69475
762,T. Young + D. Murray,NBACOMBOS,37cbb4fc-d4f4-4afd-b2b9-940715fcf818
763,P. Banchero + F. Wagner,NBACOMBOS,14a6b030-ca78-4cca-911d-9622d29290bb
764,C. Capela + D. Hunter,NBACOMBOS,37cbb4fc-d4f4-4afd-b2b9-940715fcf818


In [10]:
player_df.sport_id.unique()

array(['NFL', 'NBA', 'CFB', 'CBB', 'NHL', 'ESPORTS', 'CFL', 'NFLCOMBO',
       'NBACOMBOS', 'CFBCOMBOS'], dtype=object)

In [11]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [12]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
66,57677.0,035a13c8-8aa9-447a-99b9-917ecedc4239,0.0,4fa3580f-22bf-47c8-98f4-5c44cf56be50,0.0,Fri 11:00am,0.0,2023-11-10T16:00:00Z,regular,ESPORTS,scheduled,PUCH: LODIS vs. Orbit Anonymo,Game,2023.0
67,57684.0,cb31c28e-6b10-459b-a8e8-123f37715b0d,0.0,2822c78a-243c-4365-ab0f-bfef88cbac45,0.0,Sat 03:00pm,0.0,2023-11-11T20:00:00Z,regular,CFL,scheduled,MTL @ TOR,Game,2023.0
68,57685.0,c3cb471d-2cbd-4cb4-9d57-c8ef7b7d380b,0.0,643ebde6-4db7-48c1-844d-8aa5f8482746,0.0,Sat 06:30pm,0.0,2023-11-11T23:30:00Z,regular,CFL,scheduled,BC @ WPG,Game,2023.0


In [13]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [14]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [15]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [16]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,39443.0,17f3bc4a-e2d6-5dc5-a554-00549ff0139f,0.0,e8a4678a-ccdb-5f35-be13-aef6e59a5680,0.0,Thu 08:15pm,0.0,2023-11-09,regular,NFL,scheduled,CAR @ CHI,Game,2023.0
1,39444.0,f11f1cf1-9933-5203-8181-95020ee64399,0.0,ef876de9-81ac-5e60-af6e-0ca1700f3a51,0.0,Sun 09:30am,0.0,2023-11-12,regular,NFL,scheduled,IND @ NE,Game,2023.0
2,39450.0,7161e62b-de20-56e2-a300-0dc23637faaa,0.0,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,SF @ JAX,Game,2023.0
3,39448.0,338ae7df-02ad-5a94-9767-60f511bd55e1,0.0,01bfe9d5-f671-57ed-aa60-249fcca9267c,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,CLE @ BAL,Game,2023.0
4,39447.0,4ab08caa-b79d-598e-82ef-0906a60d2a89,0.0,ab235cf0-a041-5d36-8241-90a90f0dcb5e,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,HOU @ CIN,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,57680.0,0fc1c250-5980-4840-af9f-67b479357cb0,0.0,5e0341fa-b6e1-4718-a91d-cf1e5e993c03,0.0,Sat 12:30pm,0.0,2023-11-11,regular,ESPORTS,scheduled,BENE: Meow Gaming Club vs. A One Man Army,Game,2023.0
65,57681.0,987f333e-f93e-4837-a44f-d43f1e6bf5a2,0.0,15e82656-bb76-4985-91f7-c68fd7408d54,0.0,Sat 12:30pm,0.0,2023-11-11,regular,ESPORTS,scheduled,BENE: ZennIT vs. Mythic Esports,Game,2023.0
66,57677.0,035a13c8-8aa9-447a-99b9-917ecedc4239,0.0,4fa3580f-22bf-47c8-98f4-5c44cf56be50,0.0,Fri 11:00am,0.0,2023-11-10,regular,ESPORTS,scheduled,PUCH: LODIS vs. Orbit Anonymo,Game,2023.0
67,57684.0,cb31c28e-6b10-459b-a8e8-123f37715b0d,0.0,2822c78a-243c-4365-ab0f-bfef88cbac45,0.0,Sat 03:00pm,0.0,2023-11-11,regular,CFL,scheduled,MTL @ TOR,Game,2023.0


In [17]:
id2tm

{'e8a4678a-ccdb-5f35-be13-aef6e59a5680': 'CHI',
 'ef876de9-81ac-5e60-af6e-0ca1700f3a51': 'NE',
 'b49b653c-dc7c-516e-b1c2-da1b30d1b1a6': 'JAX',
 '01bfe9d5-f671-57ed-aa60-249fcca9267c': 'BAL',
 'ab235cf0-a041-5d36-8241-90a90f0dcb5e': 'CIN',
 '5af1e185-627e-5345-851d-3c65c5b66614': 'MIN',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 '1a20f1da-c502-5224-9c4a-bc363174cd21': 'TB',
 'c7b497d4-18b6-522b-abaa-e5c3d24bc021': 'LAC',
 '8459772c-695a-5890-afa1-7ec38da17201': 'ARI',
 '31631011-5902-52f6-ba01-c4c8d8eb3fd9': 'SEA',
 '26e67e06-664e-50a6-ad7b-c102705fde8b': 'DAL',
 '5ce78c37-b02c-52da-9e4f-fcd65b6ccb76': 'LV',
 '0719b253-43db-532e-8b5c-6c12c6c3f951': 'BUF',
 '4da4781a-a2c2-49d7-a042-1dee71740668': 'IND',
 '27c02dfc-343a-432b-8a64-c963035b609d': 'ORL',
 '402685bd-a73c-4eb1-8c0f-55debd622215': 'WAS',
 'e37df507-74a8-4a62-9430-84dc328b7480': 'LOU',
 '99b6f0b0-3970-4c2b-aba5-f18804aff3fd': 'ULL',
 '9f3952f8-b2c5-4531-bcd3-1dbd970250ff': 'SMU',
 '25cb4c5b-31af-4aa5-b200-ebf28a676cea': 'P

In [18]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [19]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Bryce Young', 'Miles Sanders', 'Chuba Hubbard', 'Adam Thielen',
       'Jonathan Mingo', 'Hayden Hurst', 'Terrace Marshall Jr.',
       'Eddy Pineiro', 'Tommy Tremble', 'Frankie Luvu', 'Derrick Brown',
       'Xavier Woods', 'Donte Jackson', 'Tyson Bagent', 'Roschon Johnson',
       "D'Onta Foreman", 'DJ Moore', 'Darnell Mooney', 'Cole Kmet',
       'Cairo Santos', 'Tyler Scott', 'Eddie Jackson', 'Jaquan Brisker',
       'Jack Sanborn', 'T.J. Edwards', 'Tyrique Stevenson',
       'Jaylon Johnson', 'Gardner Minshew II', 'Jonathan Taylor',
       'Zack Moss', 'Michael Pittman Jr.', 'Matt Gay', 'Mac Jones',
       'Rhamondre Stevenson', 'Ezekiel Elliott', 'Mike Gesicki',
       'Hunter Henry', 'Chad Ryland', 'Brock Purdy',
       'Christian McCaffrey', 'Deebo Samuel', 'Brandon Aiyuk',
       'George Kittle', 'Jauan Jennings', 'Jake Moody', 'Trevor Lawrence',
       'Travis Etienne Jr.', 'Calvin Ridley', 'Christian Kirk',
       'Evan Engram', 'Brandon McManus', 'Deshaun Watson', '

In [20]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NFL', 'NFL', 'NFL', 'NFL', 'NFL']

In [21]:
guys[1000:1007]

['Ray Davis',
 'Tayvion Robinson',
 'Dane Key',
 'Barion Brown',
 'Behren Morton',
 'Behren Morton',
 'Behren Morton']

In [22]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['17f3bc4a-e2d6-5dc5-a554-00549ff0139f',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f',
 '17f3bc4a-e2d6-5dc5-a554-00549ff0139f']

In [23]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,216.5,passing_yds,Bryce Young Passing Yards O/U,Bryce Young,NFL
1,10.5,rushing_yds,Bryce Young Rushing Yards O/U,Bryce Young,NFL
2,2.5,rushing_att,Bryce Young Rushing Attempts O/U,Bryce Young,NFL
3,33.0,passing_att,Bryce Young Passing Attempts O/U,Bryce Young,NFL
4,21.5,passing_comps,Bryce Young Completions O/U,Bryce Young,NFL
...,...,...,...,...,...
2632,39.5,pts_rebs_asts,C. Capela + D. Hunter Pts + Rebs + Asts O/U,C. Capela + D. Hunter,other
2633,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other
2634,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other
2635,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other


In [24]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,216.5,passing_yds,Bryce Young Passing Yards O/U,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
1,10.5,rushing_yds,Bryce Young Rushing Yards O/U,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
2,2.5,rushing_att,Bryce Young Rushing Attempts O/U,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
3,33.0,passing_att,Bryce Young Passing Attempts O/U,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
4,21.5,passing_comps,Bryce Young Completions O/U,Bryce Young,NFL,17f3bc4a-e2d6-5dc5-a554-00549ff0139f
...,...,...,...,...,...,...
2632,39.5,pts_rebs_asts,C. Capela + D. Hunter Pts + Rebs + Asts O/U,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818
2633,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other,J. Milton
2634,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other,J. Milton
2635,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other,J. Wright


In [25]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [26]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,216.5,passing_yds,Bryce Young Passing Yards O/U,Bryce Young,NFL,CAR,2023-11-09
1,10.5,rushing_yds,Bryce Young Rushing Yards O/U,Bryce Young,NFL,CAR,2023-11-09
2,2.5,rushing_att,Bryce Young Rushing Attempts O/U,Bryce Young,NFL,CAR,2023-11-09
3,33.0,passing_att,Bryce Young Passing Attempts O/U,Bryce Young,NFL,CAR,2023-11-09
4,21.5,passing_comps,Bryce Young Completions O/U,Bryce Young,NFL,CAR,2023-11-09
...,...,...,...,...,...,...,...
2632,39.5,pts_rebs_asts,C. Capela + D. Hunter Pts + Rebs + Asts O/U,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN
2633,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other,J. Milton,NaN
2634,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other,J. Milton,NaN
2635,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other,J. Wright,NaN


In [27]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [28]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,216.5,passing_yds,Bryce Young,NFL,CAR,2023-11-09
1,10.5,rushing_yds,Bryce Young,NFL,CAR,2023-11-09
2,2.5,rushing_att,Bryce Young,NFL,CAR,2023-11-09
3,33.0,passing_att,Bryce Young,NFL,CAR,2023-11-09
4,21.5,passing_comps,Bryce Young,NFL,CAR,2023-11-09
...,...,...,...,...,...,...
2632,39.5,pts_rebs_asts,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN
2633,3.5,passing_tds,J. Milton,other,J. Milton,NaN
2634,510.5,passing_yds,J. Milton,other,J. Milton,NaN
2635,164.5,rushing_yds,J. Wright,other,J. Wright,NaN


In [29]:
df = df.reset_index(drop=True)

In [30]:
df.Stat.unique()

array(['passing_yds', 'rushing_yds', 'rushing_att', 'passing_att',
       'passing_comps', 'passing_and_rushing_yds', 'fantasy_points',
       'receiving_yds', 'rush_rec_yds', 'receiving_rec',
       'extra_points_made', 'kicking_points', 'tackles', 'assists',
       'tackles_and_assists', 'passing_ints', 'field_goals_made',
       'passing_tds', 'rush_rec_tds', 'sacks', 'points', 'pts_rebs_asts',
       'rebounds', 'rebs_asts', 'pts_asts', 'pts_rebs', 'turnovers',
       'three_points_att', 'personal_fouls', 'blks_stls',
       'field_goals_att', 'three_points_made', 'steals', 'blocks',
       'free_throws_made', 'saves', 'shots', 'blocked_shots',
       'power_play_points', 'goals_against', 'goals', 'kills_in_map_1',
       'kills_in_maps_1_2_3', 'kills_in_map_3', 'kills_in_map_2',
       'fantasy_points_on_map_1', 'fantasy_points_on_maps_1_2_3',
       'assists_in_map_3', 'fantasy_points_on_map_2',
       'deaths_on_maps_1_2_3', 'deaths_in_map_3', 'deaths_in_map_2',
       'assists_

In [31]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [32]:
df = df.replace(syntax)

In [33]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
1613,5.5,kills_in_map_1,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
1614,6.5,kills_in_map_1,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
1615,19.5,kills_in_maps_1_2_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
1616,17.0,kills_in_maps_1_2_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
1617,5.5,kills_in_map_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
2546,10.0,kills_in_maps_1_2,pr1me,ESPORTS,Orbit Anonymo,2023-11-10
2547,8.0,kills_in_maps_1_2,Savero,ESPORTS,Orbit Anonymo,2023-11-10
2548,10.5,kills_in_maps_1_2,Roison,ESPORTS,Orbit Anonymo,2023-11-10
2549,2.0,kills_in_maps_1_2,Czypsy,ESPORTS,Orbit Anonymo,2023-11-10


In [34]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [35]:
today = dt.date.today()

In [36]:
str(today)

'2023-11-09'

In [37]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [38]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
1613,5.5,kills_in_map_1,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
1614,6.5,kills_in_map_1,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
1615,19.5,kills_in_maps_1_2_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
1616,17.0,kills_in_maps_1_2_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
1617,5.5,kills_in_map_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
2546,10.0,kills_in_maps_1_2,pr1me,ESPORTS,Orbit Anonymo,2023-11-10
2547,8.0,kills_in_maps_1_2,Savero,ESPORTS,Orbit Anonymo,2023-11-10
2548,10.5,kills_in_maps_1_2,Roison,ESPORTS,Orbit Anonymo,2023-11-10
2549,2.0,kills_in_maps_1_2,Czypsy,ESPORTS,Orbit Anonymo,2023-11-10
